In [ ]:
from comet_ml import API, APIExperiment
from comet_ml.api import Metric, Metadata, Parameter

In [ ]:
API_KEY = "fV5dK3zd6LZtaRxbPqx5dc0n0"
WORKSPACE = "real"
PROJECT_NAME = "surprise-adaptive-agents"
SAVE_DIR = "/home/ahugessen/Documents/Papers/Surprise Adapt"

In [ ]:
api = API(api_key=API_KEY)

In [ ]:
api.get(WORKSPACE)

In [ ]:
! pip install seaborn

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


### Tetris

In [ ]:
query = (Parameter("env")=='Tetris') & (Parameter("doodad_run_mode") =="local") & (Metadata("start_server_timestamp") > datetime(2023, 5, 29, 0))

In [ ]:
experiments = api.get_experiments(workspace="real", project_name="surprise-adaptive-agents", pattern="Tetris_RandomAgent_final")

In [ ]:
experiments = experiments + api.query(workspace=WORKSPACE, project_name=PROJECT_NAME, query=query)

In [ ]:
query = (Parameter("env")=='Tetris') & (Parameter("doodad_run_mode") =="local") & (Parameter("exp_name") =="SMin_final")

In [ ]:
experiments=experiments + api.query(workspace=WORKSPACE, project_name=PROJECT_NAME, query=query)

In [ ]:
query = (Parameter("env")=='Tetris') & (Parameter("doodad_run_mode") =="local") & (Parameter("exp_name") =="SMax_final")

In [ ]:
experiments=experiments + api.query(workspace=WORKSPACE, project_name=PROJECT_NAME, query=query)

In [ ]:
for experiment in experiments:
    print(experiment.get_metadata()["experimentName"])

In [ ]:
keys = [experiment.get_metadata()['experimentKey'] for experiment in experiments]
data = api.get_metrics_for_chart(keys, 
                                 metrics=["evaluation/env_infos/surprise Mean", 
                                          "evaluation/env_infos/task_reward Mean",
                                         "Epoch"],
                                parameters=['random_seed'])

In [ ]:
metrics_df = pd.DataFrame()
for key, metrics in data.items():
    experiment = api.get_experiment_by_key(experiment_key=key)
    experiment_name = experiment.get_metadata()["experimentName"]
    seed = metrics['params']['random_seed']
    for metric in metrics['metrics']:
        temp_df = pd.DataFrame(metric)
        if temp_df['metricName'][0] != 'Epoch':
            temp_df['values'] = temp_df['values'].rolling(10, center=True).mean()
        temp_df['seed'] = seed
        temp_df['experiment'] = experiment_name
        metrics_df = pd.concat([metrics_df, temp_df])

In [ ]:
metrics_df = metrics_df[['metricName', 'steps', 'seed', 'experiment', 'values']]

In [ ]:
metrics_df.head(10)

In [ ]:
metrics_df = metrics_df[metrics_df.metricName != 'Epoch'].merge(metrics_df[metrics_df.metricName=='Epoch'].drop('metricName', axis=1),
                                                   on=['steps', 'steps', 'seed', 'experiment'],
                                                  suffixes=['', '_epoch'])
metrics_df['env_steps'] = metrics_df.values_epoch*1000

In [ ]:
metrics_df.head(10)

In [ ]:
metrics_df.experiment = metrics_df.experiment.replace({'Tetris_SAv2deltabin_nomon_final': "S-Adapt-IM (Ours)",
                                                      'Tetris_SAv2deltabin_final': 'S-Adapt (Ours)',
                                                      'Tetris_SMax_final': 'S-Max',
                                                      'Tetris_SMin_final': 'S-Min',
                                                      'Tetris_RandomAgent_final': 'Random'})

In [ ]:
metrics_df.head()

In [ ]:
metrics_df.groupby(['metricName', 'seed', 'experiment']).values.count()

In [ ]:
metrics_df = metrics_df.sort_values(by="experiment")

In [ ]:
metrics_df = metrics_df[metrics_df['env_steps'] < 6500000]

In [ ]:
sns.set(font_scale=1)
ax = sns.lineplot(metrics_df[(metrics_df.metricName=='evaluation/env_infos/surprise Mean')], 
             y="values", x="env_steps", hue="experiment", palette="magma", lw=2)
leg = ax.legend()
for line in leg.get_lines():
    line.set_linewidth(4.0)
sns.move_legend(ax, "lower center", bbox_to_anchor=(.5, 1), ncol=3, title=None, frameon=False)
ax.set(xlabel="Environment Steps", ylabel='Surprise')
plt.savefig(f"{SAVE_DIR}/tetris_surprise.png")

In [ ]:
sns.set(font_scale=1)
ax = sns.lineplot(metrics_df[(metrics_df.metricName=='evaluation/env_infos/task_reward Mean')], 
             y="values", x="env_steps", hue="experiment", palette="magma", lw=2)
leg = ax.legend()
for line in leg.get_lines():
    line.set_linewidth(4.0)
sns.move_legend(ax, "lower center", bbox_to_anchor=(.5, 1), ncol=3, title=None, frameon=False)
ax.set(xlabel="Environment Steps", ylabel='Task Reward')
plt.savefig(f"{SAVE_DIR}/tetris_reward.png")

### Maze

In [ ]:
from datetime import datetime

In [ ]:
experiments = api.get_experiments(workspace="real", project_name="surprise-adaptive-agents", pattern="MazeEnvFullyObserved_SMAX_final")

In [ ]:
query = (Parameter("env") == "MazeEnvFullyObserved") & (Metadata("start_server_timestamp") > datetime(2023, 5, 28, 0))

In [ ]:
experiments = experiments + api.query(workspace=WORKSPACE, project_name=PROJECT_NAME, query=query)

In [ ]:
experiments = api.query(workspace=WORKSPACE, project_name=PROJECT_NAME, query=query)

In [ ]:
for experiment in experiments:
    print(experiment.get_metadata()["experimentName"])

In [ ]:
import pandas as pd

In [ ]:
keys = [experiment.get_metadata()['experimentKey'] for experiment in experiments]
data = api.get_metrics_for_chart(keys, 
                                 metrics=["evaluation/env_infos/surprise Mean", 
                                          "evaluation/env_infos/task_reward Mean",
                                         "Epoch"],
                                parameters=['random_seed'])

In [ ]:
metrics_df = pd.DataFrame()
for key, metrics in data.items():
    experiment = api.get_experiment_by_key(experiment_key=key)
    experiment_name = experiment.get_metadata()["experimentName"]
    seed = metrics['params']['random_seed']
    for metric in metrics['metrics']:
        temp_df = pd.DataFrame(metric)
        if temp_df['metricName'][0] != 'Epoch':
            temp_df['values'] = temp_df['values'].rolling(10, center=True).mean()
        temp_df['seed'] = seed
        temp_df['experiment'] = experiment_name
        metrics_df = pd.concat([metrics_df, temp_df])

In [ ]:
metrics_df = metrics_df[['metricName', 'steps', 'seed', 'experiment', 'values']]

In [ ]:
metrics_df = metrics_df[metrics_df.metricName != 'Epoch'].merge(metrics_df[metrics_df.metricName=='Epoch'].drop('metricName', axis=1),
                                                   on=['steps', 'steps', 'seed', 'experiment'],
                                                  suffixes=['', '_epoch'])
metrics_df['env_steps'] = metrics_df.values_epoch*1000

In [ ]:
metrics_df.groupby(['metricName', 'seed', 'experiment']).values.count()

In [ ]:
metrics_df.head()

In [ ]:
metrics_df.experiment = metrics_df.experiment.replace({'MazeEnvFullyObserved_SA_no_momentum_deltabin': "S-Adapt-IM",
                                                      'MazeEnvFullyObserved_SA_momentum_deltabin_final': 'S-Adapt',
                                                      'MazeEnvFullyObserved_SMax_final': 'S-Max',
                                                      'MazeEnvFullyObserved_SMin_final': 'S-Min',
                                                      'MazeEnvFullyObserved_RandomAgent_final': 'Random'})

In [ ]:
metrics_df = metrics_df.sort_values(by="experiment")

In [ ]:
sns.set(font_scale=1)
ax = sns.lineplot(metrics_df[(metrics_df.metricName=='evaluation/env_infos/surprise Mean')], 
             y="values", x="env_steps", hue="experiment", palette="magma", lw=2)
leg = ax.legend()
for line in leg.get_lines():
    line.set_linewidth(4.0)
sns.move_legend(ax, "lower center", bbox_to_anchor=(.5, 1), ncol=3, title=None, frameon=False)
ax.set(xlabel="Environment Steps", ylabel='Surprise')
plt.savefig(f"{SAVE_DIR}/maze_surprise.png")

In [ ]:
sns.set(font_scale=1)
ax = sns.lineplot(metrics_df[(metrics_df.metricName=='evaluation/env_infos/task_reward Mean')], 
             y="values", x="env_steps", hue="experiment", palette="magma", lw=2)
ax.set(xlabel="Environment Steps", ylabel='Task Reward')
leg = ax.legend()
for line in leg.get_lines():
    line.set_linewidth(4.0)
sns.move_legend(ax, "lower center", bbox_to_anchor=(.5, 1), ncol=3, title=None, frameon=False)
plt.savefig(f"{SAVE_DIR}/maze_reward.png")